In [1]:
# Cell 1: Install libraries
!pip install mediapipe opencv-python pandas scikit-learn numpy matplotlib

In [2]:
# Cell 2: Libraries
import mediapipe as mp
import cv2
import numpy as np
import time
import pandas as pd
import csv
import os
from matplotlib import pyplot as plt

# Mediapipe initialization and Video recording

In [ ]:
# Cell 3: Check that the camera and mediapipe are working
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                print("Error: Unable to read frame from video capture")
                break

            # Flip image to simulate mirror view
            image = cv2.flip(image, 1)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            pose_results = pose.process(image)
            hand_results = hands.process(image)

            # RGB 2 BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw points
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                )

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                    )

            cv2.imshow("Raw Webcam Feed", image)

            if cv2.waitKey(10) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Cell 4: Record video for data capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30 
    print(f"Video Resolution: {width}x{height} at {fps} FPS")

    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Error: Unable to read frame from video capture")
            break
        frame = cv2.flip(frame, 1)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)  
        frame_rgb.flags.writeable = True
        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        # Write every frame
        out.write(frame)

        cv2.imshow('MediaPipe Hands', frame)  
        if cv2.waitKey(5) & 0xFF == ord('q'): 
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
# Cell 5: Create CSV to store data
headers = ['class', 'accuracy', 'sequence'] 
headers.extend([f'pose_{coord}{i}' for i in range(33) for coord in ('x', 'y', 'z', 'v')])
headers.extend([f'{hand}_{coord}{i}' for hand in ('right_hand', 'left_hand') for i in range(21) for coord in ('x', 'y', 'z', 'v')])

with open('coordinates_1.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(headers)

In [ ]:
# Cell 6: Capture data for first movement
cap = cv2.VideoCapture('output_paint.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_1.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            # Select the sequence number based on the key pressed
            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_1.csv')
df.loc[df['accuracy'].notna(), 'class'] = 'pen'
df.to_csv('coordinates_1.csv', index=False)


In [ ]:
# Cell 7: Capture data for second movement
cap = cv2.VideoCapture('output_paint.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_2.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file, lineterminator='\n')

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_2.csv', header=None)
df.loc[df[1].notna(), 0] = 'paint'
df.to_csv('coordinates_2.csv', header=False, index=False)


In [ ]:
# Cell 7.2: Capture data for third movement
cap = cv2.VideoCapture('output_scissors.avi')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequences = {'r': -1, 'w': -1}  
    recording_state = None

    with open('coordinates_3.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file, lineterminator='\n')

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            current_sequence = sequences[recording_state] if recording_state else -1
            row = ['', accuracy, current_sequence]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF

            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequences[new_state] += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

df = pd.read_csv('coordinates_3.csv', header=None)
df.loc[df[1].notna(), 0] = 'scissors'
df.to_csv('coordinates_3.csv', header=False, index=False)


# Data Processing

In [ ]:
# Cell 8: Join all the CSVs of the different movements
df1 = pd.read_csv('coordinates_1.csv', header=None)
df2 = pd.read_csv('coordinates_2.csv', header=None)
df3 = pd.read_csv('coordinates_3.csv', header=None)


combined_df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
combined_path = 'combined_coordinates.csv'
combined_df.to_csv(combined_path, header=False, index=False)

In [3]:
# Cell 9: Delete not important data
data = pd.read_csv('combined_coordinates.csv')

# Identify columns to remove: include specific ranges and all visibility points
###columns_to_remove_1 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(0, 11)]
###columns_to_remove_2 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(23, 33)]
visibility_columns_pose = [col for col in data.columns if 'pose_v' in col]
visibility_columns_left_hand = [col for col in data.columns if 'left_hand_v' in col]
visibility_columns_right_hand = [col for col in data.columns if 'right_hand_v' in col]

# Combine all columns to remove
###columns_to_remove = columns_to_remove_1 + columns_to_remove_2 + visibility_columns_pose + visibility_columns_left_hand + visibility_columns_right_hand
columns_to_remove = visibility_columns_pose + visibility_columns_left_hand + visibility_columns_right_hand
columns_to_remove = [col for col in columns_to_remove if col in data.columns]

# Drop the selected columns from the dataframe
data_filtered = data.drop(columns=columns_to_remove)

# Round the values and save the filtered data
###data_filtered = data_filtered.round(3)
data_filtered.to_csv('filtered_coordinates.csv', index=False)

# Display the first few rows of the filtered dataset
data_filtered.head()

,class,accuracy,sequence,pose_x0,pose_y0,pose_z0,pose_x1,pose_y1,pose_z1,pose_x2,...,left_hand_z17,left_hand_x18,left_hand_y18,left_hand_z18,left_hand_x19,left_hand_y19,left_hand_z19,left_hand_x20,left_hand_y20,left_hand_z20
0,pen,1,0,0.411205,0.171439,-0.444944,0.427558,0.101079,-0.442622,0.442020,...,-0.034374,0.348000,0.782419,-0.050452,0.351776,0.811713,-0.058446,0.355583,0.840286,-0.062885
1,pen,1,0,0.409809,0.172871,-0.436721,0.425809,0.101436,-0.434942,0.440617,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,pen,1,0,0.408941,0.176701,-0.447924,0.424350,0.102597,-0.446523,0.439302,...,-0.041063,0.350450,0.785777,-0.057279,0.356252,0.815158,-0.064492,0.362689,0.842664,-0.068444
3,pen,1,0,0.408325,0.181097,-0.382025,0.423258,0.104253,-0.384934,0.438338,...,-0.051957,0.354071,0.783193,-0.066484,0.361161,0.810169,-0.070669,0.369577,0.833266,-0.072963
4,pen,1,0,0.406662,0.182538,-0.425791,0.420278,0.104820,-0.424994,0.435957,...,-0.052415,0.352663,0.778692,-0.069039,0.359794,0.805952,-0.075388,0.368541,0.831612,-0.079067


In [7]:
def transform_and_merge_columns(file_path, output_file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    data['accuracy'] = data['accuracy'].map({0: 'W', 1: 'R'})
    data['label'] = data.iloc[:, 0].astype(str) + '_' + data['accuracy']
    
    label_column = data.pop('label') 
    data.insert(0, 'label', label_column)
    
    data.drop(columns=['class', 'accuracy'], inplace=True)
    data.to_csv(output_file_path, index=False)

input_file_path = 'filtered_coordinates.csv'  
output_file_path = 'modified_file.csv'  
transform_and_merge_columns(input_file_path, output_file_path)

In [ ]:
# Cell 10: Create folders in directory
data = pd.read_csv('filtered_coordinates.csv')

base_dir = 'DataBase'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
unique_classes = data['class'].unique()

for class_value in unique_classes:
    class_dir = os.path.join(base_dir, str(class_value))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
    
    class_data = data[data['class'] == class_value]

    for accuracy_type, subfolder_name in zip([1, 0], ['R', 'W']):
        accuracy_dir = os.path.join(class_dir, subfolder_name)
        if not os.path.exists(accuracy_dir):
            os.makedirs(accuracy_dir)
        
        accuracy_data = class_data[class_data['accuracy'] == accuracy_type]
        
        unique_sequences = accuracy_data['sequence'].unique()
        
        for sequence in unique_sequences:
            sequence_dir = os.path.join(accuracy_dir, str(sequence))
            if not os.path.exists(sequence_dir):
                os.makedirs(sequence_dir)

In [ ]:
# Cell 11: Save individual np arrays
data = pd.read_csv('filtered_coordinates.csv')
base_dir = 'DataBase'

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

for class_value in data['class'].unique():
    class_dir = os.path.join(base_dir, str(class_value))
    class_data = data[data['class'] == class_value]

    for accuracy_type, subfolder_name in zip([1, 0], ['R', 'W']):
        accuracy_dir = os.path.join(class_dir, subfolder_name)
        accuracy_data = class_data[class_data['accuracy'] == accuracy_type]

        for sequence in accuracy_data['sequence'].unique():
            sequence_dir = os.path.join(accuracy_dir, str(sequence))
            sequence_data = accuracy_data[accuracy_data['sequence'] == sequence]

            sequence_data = sequence_data.reset_index(drop=True)

            for index, row in sequence_data.iterrows():
                frame_path = os.path.join(sequence_dir, f'{index}.npy')
                np.save(frame_path, row.values[3:])

print("cosas")

In [ ]:
# Cell 12: Test if array is in the correct way
np.load('DataBase/scissors/W/4/0.npy', allow_pickle=True)

In [ ]:
array=np.load('DataBase/movement_1/W/0/0.npy', allow_pickle=True)
len(array)

# Prepare data for training

In [ ]:
# Cell 13: Install new libraries for processing and training
!pip install tensorflow 
!pip install tensorflow-macos
!pip install tensorflow-metal
!pip install numpy --upgrade
!pip install pandas --upgrade
!pip install matplotlib --upgrade
!pip install scikit-learn --upgrade
!pip install scipy --upgrade
!pip install plotly --upgrade

In [ ]:
!conda install -y apple tensorflow-deps
!conda install notebook -y

In [ ]:
import sys
import keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print (f"Python Platform: {platform.platform ()}")
print (f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print ()

print (f"Python {sys.version}")
print (f"Pandas {pd.__version__}")
print (f"Scikit-Learn {sk.__version__}")
print (f"SciPy {sp.__version__}")
gpu = len (tf.config.list_physical_devices ('GPU'))>0
print ("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
# Cell 14: Import new libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Cell 15: Joining and labeling arrays
base_dir = 'DataBase'
features = []
labels = []

for class_name in os.listdir(base_dir):
    class_dir = os.path.join(base_dir, class_name)
    for accuracy_type in ['R', 'W']:
        accuracy_dir = os.path.join(class_dir, accuracy_type)
        for sequence in os.listdir(accuracy_dir):
            sequence_dir = os.path.join(accuracy_dir, sequence)
            sequence_features = []
            for file_name in os.listdir(sequence_dir):
                file_path = os.path.join(sequence_dir, file_name)
                data = np.load(file_path, allow_pickle=True)
                sequence_features.append(data)
            features.append(sequence_features)
            labels.append(f"{class_name}_{accuracy_type}")

features_padded = pad_sequences(features, padding='post', dtype='float32')

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# Cell after 15: Extract and print unique labels before encoding
# This code assumes 'labels' list is already populated as in Cell 15

# Extract unique labels
unique_labels = set(labels)

# Print unique labels
print("Unique labels before encoding:")
print(unique_labels)

# Optionally, if you want to save these unique labels to a text file:
with open('unique_labels.txt', 'w') as f:
    for label in sorted(unique_labels):  # Sorting to maintain consistent order
        f.write("%s\n" % label)


In [ ]:
unique_labels

In [ ]:
# Cell 16: Labels test (features_padded and labels_encoded) outputs expected in .shape f_p(x, y, z) / l_e(x, )
np.array(labels).shape

In [ ]:
# Cell 17: Set X and y
X = np.array(features_padded)
y = to_categorical(labels_encoded).astype(int)
X

In [ ]:
import numpy as np

# Assume X is your existing NumPy array
# Example: X = np.random.rand(10, 10, 10) - Replace with your actual array

# Set numpy print options to avoid truncation
np.set_printoptions(threshold=np.inf)

# Convert the array X to string
array_str = np.array2string(X)

# Write the string to a file
with open('X.txt', 'w') as f:
    f.write(array_str)


In [ ]:
with open('y.txt', 'w') as file:
    # Convert the list into a string and write it to the file
    file.write(str(y))

In [ ]:
# Cell 18: Split values for testing and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_train.shape

# Build model + LSTM training

In [ ]:
# Cell 19: Import training dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Cell 20: To web monitor live training later
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
num_classes = 6  # Put here amount of movements you want to detect
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(68, 162)))  # Put here las two values of 'X.shape' in this case X.shape = (122, 68, 162)
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh')) 
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax')) 

In [ ]:
res = [0.7, 0.3, 0.5, 0.4, 0.3, 0.2]

In [ ]:
features_padded[np.argmax(res)]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
# To keep track of the training in your terminal cd to Logs/train folder and then write tensorboard --logdir=. a link will pop up that you just need to put in your search engine

In [ ]:
model.summary()

# Make predictions

In [ ]:
res=model.predict(X_test)

In [ ]:
np.argmax(res[0])

In [ ]:
# Assume you have a single set of features to predict
model_output = model.predict(np.expand_dims(features_padded[100], axis=0))  
predicted_index = np.argmax(model_output[0])  
predicted_label = label_encoder.inverse_transform([predicted_index])[0]  

print("Predicted Label:", predicted_label)  


In [ ]:
model.save('actions.h5')

In [ ]:
model.summary()

# Model evaluation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()  
yhat = np.argmax(yhat, axis=1).tolist()  

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# Real time predictions

In [ ]:
# Read the contents of the text file
with open('unique_labels.txt', 'r') as file:
    actions = [line.strip() for line in file.readlines()]

# Print the list of actions
actions

In [ ]:
import mediapipe as mp
import cv2
import numpy as np

# Number of frames to collect data for
num_frames_to_collect = 68

# Initialize an array to store the collected data
collected_data = []

# Setup MediaPipe instances
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
         mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2) as hands:
        frame_count = 0
        while cap.isOpened() and frame_count < num_frames_to_collect:
            ret, image = cap.read()
            if not ret:
                print("Error: Unable to read frame from video capture")
                break

            # Flip image to simulate mirror view
            image = cv2.flip(image, 1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            pose_results = pose.process(image)
            hand_results = hands.process(image)

            # Flatten pose landmarks, skipping certain indices
            if pose_results.pose_landmarks:
                pose_landmarks = [
                    round(value, 3) for idx, landmark in enumerate(pose_results.pose_landmarks.landmark)
                    if not (0 <= idx <= 11 or 23 <= idx <= 32)  # Adjust to 0-based indexing and include index 32
                    for value in (landmark.x, landmark.y, landmark.z)
                ]
            else:
                pose_landmarks = [0] * (10 * 3)  # Adjust count to reflect the remaining landmarks

            # Initialize hand landmarks placeholders
            right_hand_landmarks = [0] * (21 * 3)
            left_hand_landmarks = [0] * (21 * 3)

            # Detect and sort hand landmarks
            if hand_results.multi_hand_landmarks:
                hand_labels = [hand.classification[0].label for hand in hand_results.multi_handedness]
                for hand_landmarks, label in zip(hand_results.multi_hand_landmarks, hand_labels):
                    flat_hand = [round(value, 3) for landmark in hand_landmarks.landmark
                                 for value in (landmark.x, landmark.y, landmark.z)]
                    if label == 'Right':
                        right_hand_landmarks = flat_hand
                    else:
                        left_hand_landmarks = flat_hand

            # Combine all landmarks
            all_landmarks = pose_landmarks + right_hand_landmarks + left_hand_landmarks
            collected_data.extend(all_landmarks)  # Append to flat list

            # Drawing
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                )

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                    )

            cv2.imshow("Raw Webcam Feed with Landmarks", image)

            # Increment frame count
            frame_count += 1
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# Convert to numpy array
collected_data_array = np.array(collected_data)
print("Data shape:", collected_data_array.shape)


In [ ]:
# Put time to wait
# Import model
# make predictions straight to matrix 
# Convert matrix results into labels

In [ ]:
from tensorflow.keras.models import load_model

# Load your model
model = load_model('actions.h5')
model.summary()

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load your model
model = load_model('actions.h5')

# Example data for one sequence
# Assume 'sequence' is a list of arrays, with each array representing frame data which is then flattened
sequence = collected_data_array  # Replace these with actual frame data

# If your model expects a 2D array (batch size, features):
sequence = np.expand_dims(sequence, axis=0)

# Predict
predictions = model.predict(sequence)
print(predictions)


In [ ]:
collected_data_array